In [32]:
import os
import sys
import time
import json

import urllib
import urllib.request
import requests

import pandas as pd
from bs4 import BeautifulSoup

import re

from tqdm import tqdm

In [10]:
data_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieInfo\hot"
save_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieReview"
# data_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieReview"
# data_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieReview"

In [13]:
movies = pd.read_csv(os.path.join(data_path, "movies_hot_merged_en.csv"), encoding="euc-kr")

In [15]:
movies

,movieCd,movieNm,movieNmEn,prdtYear,openDt,prdtStatNm,repGenre,genre,repNation,nations,...,directors,actors,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm
0,20090834,아바타,Avatar,2009,20091217,개봉,SF,"SF,액션,어드벤처",미국,미국,...,제임스 카메론,"['샘 워싱턴', '조 샐다나', '시고니 위버']","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...","['', '', '']","['', '', '']","['20101507', '20175001', '20100041', '20175001...","['이십세기폭스필름코퍼레이션', '주식회사 해리슨앤컴퍼니', '이십세기폭스코리아(주...","['20th Century Fox Film Corporation', 'Harriso...","['제작사', '배급사', '배급사', '수입사', '수입사']","['12세이상관람가', '12세이상관람가']"
1,19980074,타이타닉,Titanic,1997,19980220,개봉,드라마,"드라마,멜로/로맨스,액션",미국,미국,...,제임스 카메론,"['레오나르도 디카프리오', '케이트 윈슬렛']","['Leonardo Dicaprio', 'Kate Winslet']","['', '']","['', '']","['20104469', '20040021', '20100041', '20142849...","['파라마운트 픽쳐스', '씨네힐', '이십세기폭스코리아(주)', '(주)영화사오원...","['Paramount Pictures', 'CINEHILL', '20th Centu...","['제작사', '배급사', '배급사', '수입사', '수입사']","['15세 미만인 자는 관람할 수 없는 등급 ', '15세이상관람가']"
2,20129370,명량,Roaring Currents,2013,20140730,개봉,사극,"사극,액션",한국,한국,...,김한민,"['최민식', '류승룡', '조진웅', '김명곤', '진구', '이정현', '권율'...","['CHOI Min-shik', 'RYU Seung-ryong', 'CHO Jin-...","['이순신 ', '구루지마 ', '와키자카 ', '도도', '임준영 ', '정씨 여...","['', '', '', '', '', '', '', '', '', '', '', '...","['20123063', '20110854', '20110854', '20100109']","['(주)빅스톤픽쳐스', '씨제이이앤엠(주)', '씨제이이앤엠(주)', '씨제이엔터...","['BIGSTONEPICTURES', 'CJ ENM Corp.', 'CJ ENM C...","['제작사', '배급사', '제공', '해외세일즈사']",['15세이상관람가']
3,20124079,"광해, 왕이 된 남자",Masquerade,2012,20120913,개봉,사극,"사극,드라마",한국,한국,...,추창민,"['이병헌', '류승룡', '한효주', '장광', '김인권', '심은경', '김명곤...","['LEE Byung-hun', 'RYU Seung-ryong', 'HAN Hyo-...","['광해/하선', '허균', '중전', '조내관', '도부장', '사월이', '박충...","['', '', '', '', '', '', '', '', '', '', '', '...","['20100540', '20110854', '20110854', '20110854...","['리얼라이즈픽쳐스(주)', '씨제이이앤엠(주)', '씨제이이앤엠(주)', '씨제이...","['Realies Pictures, Inc.', 'CJ ENM Corp.', 'CJ...","['제작사', '제작사', '배급사', '제공', '공동제공', '공동제공', '공...",['15세이상관람가']
4,20070061,트랜스포머,Transformers,2007,20070628,개봉,SF,"SF,액션,어드벤처",미국,미국,...,마이클 베이,"['샤이아 라보프', '메간 폭스']","['Shia LaBeouf', 'Megan Fox']","['샘 윗윅키', '']","['', '']","['20100694', '20102885', '20104469', '20100109...","['드림웍스픽쳐스', '드림웍스 SKG', '파라마운트 픽쳐스', '씨제이엔터테인먼...","['', 'DreamWorks SKG ', 'Paramount Pictures', ...","['제작사', '제작사', '제작사', '배급사', '수입사']",['12세이상관람가']
5,20184889,어벤져스: 엔드게임,Avengers: Endgame,2018,20190424,개봉,액션,"액션,SF",미국,미국,...,안소니 루소,"['로버트 다우니 주니어', '크리스 에반스', '마크 러팔로', '크리스 헴스워스...","['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf...","['', '', '', '', '', '', '', '', '', '', '', '']","['', '', '', '', '', '', '', '', '', '', '', '']","['20161801', '20161801']","['월트디즈니컴퍼니코리아 유한책임회사', '월트디즈니컴퍼니코리아 유한책임회사']","['The Walt Disney Company Korea', 'The Walt Di...","['배급사', '수입사']","['12세이상관람가', '12세이상관람가']"
6,20183867,알라딘,Aladdin,2018,20190523,개봉,어드벤처,"어드벤처,가족,판타지",미국,미국,...,가이 리치,"['메나 마수드', '윌 스미스', '나오미 스콧']","['Mena Massoud', 'Will Smith', 'Naomi Scott']","['', '', '']","['', '', '']","['20161801', '20161801']","['월트디즈니컴퍼니코리아 유한책임회사', '월트디즈니컴퍼니코리아 유한책임회사']","['The Walt Disney Company Korea', 'The Walt Di...","['배급사', '수입사']","['전체관람가', '전체관람가']"
7,20127593,7번방의 선물,Miracle in Cell No.7,2012,20130123,개봉,코미디,"코미디,드라마",한국,한국,...,이환경,"['류승룡', '박신혜', '갈소원', '정진영', '오달수', '박원상', '김정...","['RYU Seung-ryong', 'PARK Shin-hye', 'GAL So-w...","['용구', '큰 예승', '어린 예승', '특별출연-민환', '방장', '춘호',...","['', '', '', '', '', '', '', '', '', '', '', '...","['20100411', '20100934', '20100932', '20100932...","['(주)화인웍스', '(주)씨엘엔터테인먼트', '(주)넥스트엔터테인먼트월드(NEW...","['FineWorks', 'CL entertainment, Inc.', 'Next ...","['제작사', '제작사', '배급사', '제공', '공동제공', '공동제공', '공...",['15세이상관람가']
8,20136803,겨울왕국,Frozen,2013,20140116,개봉,애니메이션,"애니메이션,어드벤처,가족,코미디,뮤지컬,판타지",미국,미국,...,크리스 벅,"['이디나 멘젤', '크리스틴 벨', '조나단 그로프', '조시 게드']","['Idina Menzel', 'Kristen Bell', 'Jonathan Gro...","['엘사 목소리 ', '안나 목소리 ', '크리스토프 목소리 ', '올라프 목소리 ']","['', '', '', '']","['20125636', '20125636']","['소니픽쳐스릴리징월트디즈니스튜디오스코리아(주)', '소

In [16]:
def progress_bar(i, total):
    k = i/total*100
    star = "*"*int(k/2.5)
    print("Processing: {}/{} [{:.1f}%] {}".format(i, total, k, star), end="\r")

In [37]:
def get_naverApi(movieNm, openDt, director):
    
    client_id = "JjcGyzzf5rjyFKNPCTxG"
    client_secret = "MDi_1o76AT"
    encText = urllib.parse.quote(f"{movieNm}")
    url = "https://openapi.naver.com/v1/search/movie?query=" + encText + f"&yearfrom={openDt-1}&yearto={openDt}"
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode == 200):
        html = response.read().decode("utf-8")
        parse = json.loads(html)
                
        if parse["items"] == []:
            link = ""
            
            
        else:
            if len(parse["items"]) > 1:
                for i, item in enumerate(parse["items"]):
                    if director in item["director"]:
                        idx = i
                    elif item["pubDate"] == str(openDt):
                        idx = i
                    else:
                        idx = 0
            else:
                 idx = 0
            
            info = parse["items"][idx]
            link = info["link"]
            
    else:
        link = ""
    
    return movieNm, link

In [38]:
def load_naverMovie(movie_list):
    movie_list.openDt = movie_list.openDt.astype(str)
    ls = movie_list[["movieCd","movieNm", "openDt", "directors"]].values
    
    i = 1
    total = len(ls)
    links = []
    movieNms = []
    movieCds = []
    for v in ls:
        movieNm, link = get_naverApi(v[1], int(v[2][:4]), v[3].split(" ")[0].split(",")[0])
        code = v[0]
        links.append(link)
        movieNms.append(movieNm)
        movieCds.append(code)
        progress_bar(i, total)
        i += 1
        time.sleep(0.1)

    print()
    print("complete!")
    
    return movieCds, movieNms, links

In [19]:
movieNm, link = get_naverApi("타이타닉", 1997, "제임스 카메론")

In [21]:
movieNm, link

('타이타닉', 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=18847')

In [121]:
naver_movie_code = link[51:]

In [39]:
def get_reviews(naver_movie_code):
    target_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={naver_movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "html.parser")
    review_total = soup.find("div",class_="score_total")
    
#     review가 없는 경우는 아무것도 리턴하지 않는다
    if review_total is not None:
        number = review_total.find('em').text
        number = int(re.sub(',', '', number))
        page_num = number //10

        user_id = []
        review = []
        score = []
        
        for page in range(1, page_num):
            target_url_page = target_url + '&page={}'.format(page)
            soup = BeautifulSoup(urllib.request.urlopen(target_url_page).read(), "html.parser")
            review_divs = soup.find_all('div',class_='score_reple')
            score_divs = soup.find_all('div', class_="star_score")
            for spans in review_divs:
                txt = spans.find('p').text.strip()
                if txt is not None:
                    review.append(txt)
                else:
                    review.append('')
            for spans in review_divs:
                dl = spans.find('dl')
                if dl is not None:
                    user = dl.find('span').text
                    user_id.append(user)
                else:
                    user_id.append('')
            for span in score_divs:
                if span is not None:
                    s = span.find('em').text.strip()
                    score.append(s)
                else:
                    score.append('')
            
        return user_id, review, score
    else:
        return

In [25]:
movieCds, movieNms, links = load_naverMovie(movies)

Processing: 20/20 [100.0%] ****************************************
complete!


In [36]:
links[0][51:]

'62266'

In [ ]:
user_id = []
review = []
score = []
for link in tqdm(links):
    code = link[51:]
    ids, reply, star_score = get_reviews(code)
    user_id.append(ids)
    review.append(reply)
    score.append(star_score)

 25%|███████████████████▌                                                          | 5/20 [1:23:07<3:44:22, 897.52s/it]

In [ ]:
user_id, review, score = get_reviews(movies)

In [114]:
titanic = pd.DataFrame([user_id, review, score], index=['user_id', 'review', 'score']).T

In [118]:
titanic.to_csv(os.path.join(save_path,'titanic.csv'), encoding="utf-8")

In [13]:
def main():
    movieCds, movieNms, links = load_naverMovie(movies)
    movie_dicts = dict()
    for movieNm, link in zip(movieNms,links):
        movie_code = link[51:]
        user_id, review, score = get_review(movie_code)
        if review is not None:
            movie_dicts[movieNm].append(review)
    
    movie_review_df = pd.DataFrame.from_dict(movie_dicts, orient='index').T
    return movie_review_df

In [14]:
movie_review_df = main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-f6598aa97c5d>", line 1, in <module>
    movie_review_df = main()
  File "<ipython-input-13-459aadcb67ef>", line 2, in main
    movieNms, links = load_naverMovie(movies)
  File "<ipython-input-6-23ac7fc798ca>", line 16, in load_naverMovie
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2039, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython

KeyboardInterrupt: 